In [1]:
import json
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
# import glmnet_python
# from glmnet_python import glmnet
# from glmnet_python import glmnetPredict
# from glmnet_python import glmnetPlot
# from glmnet_python import cvglmnet
# from glmnet_python import cvglmnetPredict
# from glmnet_python import cvglmnetPlot
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import codecs

In [2]:
# VARIABLES TO ADJUST
# 'Mugs', 'Plates', 'Geometric', 'Cutlery', 'Ball'
family_to_select = 'Ball'
num_bins = 20
cv = 3

In [3]:
# READING FILES AND CREATING DATASET
file_data = "./PyData/filtered_data.json"
file_eps = "./PyData/ep_labels.json"
file_task = "./PyData/task_labels.json"
file_allSyn = "./PyData/all_subjects_scores.json"
file_emg = "./PyData/emg_data.json"

with open(file_data, "r") as f:
    data = json.load(f)  # data[subjects][trials][joints]

with open(file_eps, "r") as g:
    eps = json.load(g)  # eps[subjects][trials]

with open(file_task, "r") as h:
    task = json.load(h)  # task[subjects][trials]

with open(file_allSyn, "r") as h:
    all_syn = json.load(h)  # all_syn[subjects X trials][joints]
    
with open(file_emg, "r") as f:
    emg_data = json.load(f)  # data[subjects][trials][sensors]

In [4]:
# VECTORIZING DATA

vectorized_task = [x for sublist in task for x in sublist]  # Vectorization of tasks
vectorized_eps = [x for sublist in eps for x in sublist]  # Vectorization of eps
vectorized_data = [x for sublist in data for x in sublist]  # Vectorization of trials
vectorized_data = np.array(vectorized_data, dtype=float) # Conversion to float to we replace 'None' with 'NaN'
vectorized_emg_data = [x for sublist in emg_data for x in sublist]  # Vectorization of trials
vectorized_emg_data = np.array(vectorized_emg_data, dtype=float) # Conversion to float to we replace 'None' with 'NaN'
given_object = [x.split("_")[0] for x in vectorized_task]  # Vectorized given objects
ask_object = [x.split("_")[1] for x in vectorized_task]  # Vectorized asked objects
all_syn = np.array(all_syn, dtype=float) # Conversion to float to replace 'None' with 'NaN'

In [5]:
# SPLIT BY TRIALS

tr_idx = [index for index, _ in enumerate(vectorized_task) if vectorized_task[index] != vectorized_task[index-1]]
tr_idx.append(len(vectorized_task))

# all these lists are [trials]x[timepoints per trial]
spl_task = [vectorized_task[tr_idx[i]:tr_idx[i+1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx)-1]
spl_eps = [vectorized_eps[tr_idx[i]:tr_idx[i + 1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx) - 1]
spl_dat = [vectorized_data[tr_idx[i]:tr_idx[i + 1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx) - 1]
spl_emg = [vectorized_emg_data[tr_idx[i]:tr_idx[i + 1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx) - 1]
spl_given = [given_object[tr_idx[i]:tr_idx[i + 1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx) - 1]
spl_ask = [ask_object[tr_idx[i]:tr_idx[i + 1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx) - 1]
spl_syn = [all_syn[tr_idx[i]:tr_idx[i + 1]] for i, _ in enumerate(tr_idx) if i != len(tr_idx) - 1]


In [6]:
# SELECT TRIALS BY FAMILY

obj_fam = dict(
    CeramicMug = 'Mugs',
    Glass = 'Mugs',
    MetalMug = 'Mugs',
    CeramicPlate = 'Plates',
    MetalPlate = 'Plates',
    PlasticPlate = 'Plates',
    Cube = 'Geometric',
    Cylinder ='Geometric',
    Triangle ='Geometric',
    Fork = 'Cutlery',
    Knife ='Cutlery',
    Spoon ='Cutlery',
    PingPongBall = 'Ball',
    SquashBall='Ball',
    TennisBall='Ball'
)

fam_idx = list()
for it in range(len(spl_given)):
    if obj_fam[spl_given[it][0]] == family_to_select:
        fam_idx.append(it)

selected_task = [spl_task[idx] for idx in fam_idx]
selected_eps = [spl_eps[idx] for idx in fam_idx]
selected_dat = [spl_dat[idx] for idx in fam_idx]
selected_emg = [spl_emg[idx] for idx in fam_idx]
selected_given = [spl_given[idx] for idx in fam_idx]
selected_ask = [spl_ask[idx] for idx in fam_idx]
selected_syn = [spl_syn[idx] for idx in fam_idx]

In [7]:
# DIVIDE BY EPs

# all these list are [trials]x[eps per trial]x[timepoints per ep]
final_task = list()
final_eps = list()
final_given = list()
final_ask = list()
# this lists are [trials]x[eps per trial]x[bins per ep]x[joints]
final_dat = list()
final_emg = list()
final_syn = list()

for e in range(len(selected_eps)):

    ch_idx = [index for index, _ in enumerate(selected_eps[e]) if selected_eps[e][index] != selected_eps[e][index-1]]
    ch_idx.append(len(selected_eps[e]))
    if 0 not in ch_idx:
        ch_idx.insert(0, 0)

    sel_task = [selected_task[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    final_task.append(sel_task)
    sel_eps = [selected_eps[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    final_eps.append(sel_eps)
    sel_given = [selected_given[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    final_given.append(sel_given)
    sel_ask = [selected_ask[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    final_ask.append(sel_ask)

    sel_dat = [selected_dat[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    sel_emg = [selected_emg[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    sel_syn = [selected_syn[e][ch_idx[i]:ch_idx[i + 1]] for i, _ in enumerate(ch_idx) if i != len(ch_idx) - 1]
    
    # DIVIDE BY BINS
    aux = list()
    aux_syn = list()
    aux_emg = list()
    for j in range(len(sel_dat)):
        div_dat = np.array_split(sel_dat[j], num_bins)
        div_emg = np.array_split(sel_emg[j], num_bins)
        div_syn = np.array_split(sel_syn[j], num_bins)
        
        dat_mean = [np.nanmean(x, axis=0) for x in div_dat]
        syn_mean = [np.nanmean(x, axis=0) for x in div_syn]
        emg_mean = [np.nanmean(x, axis=0) for x in div_emg]
        
        aux.append(dat_mean)
        aux_syn.append(syn_mean)
        aux_emg.append(emg_mean)
      
    final_dat.append(aux)
    final_syn.append(aux_syn)
    final_emg.append(aux_emg)

/tmp/ipykernel_9297/3717972689.py:42: RuntimeWarning: Mean of empty slice
  dat_mean = [np.nanmean(x, axis=0) for x in div_dat]
/tmp/ipykernel_9297/3717972689.py:43: RuntimeWarning: Mean of empty slice
  syn_mean = [np.nanmean(x, axis=0) for x in div_syn]
/tmp/ipykernel_9297/3717972689.py:44: RuntimeWarning: Mean of empty slice
  emg_mean = [np.nanmean(x, axis=0) for x in div_emg]


In [8]:
# SELECT TRIALS CV

total_acc = list()
unique_given = [final_given[i][0][0] for i in range(len(final_given))]  # unique object per trial
skf = StratifiedKFold(n_splits=cv)
for train, test in skf.split(final_dat, unique_given):

    train_given = [unique_given[x] for x in train]
    test_given = [unique_given[y] for y in test]
    train_data = [final_dat[x] for x in train]
    test_data = [final_dat[y] for y in test]
    train_emg = [final_emg[x] for x in train]
    test_emg = [final_emg[y] for y in test]
    train_syn = [final_syn[x] for x in train]
    test_syn = [final_syn[y] for y in test]

    # FOR SINGLE SOURCE CLASSIFICATION
    trn_dat = list()
    trn_emg = list()
    trn_lab = list()
    trn_syn = list()

    for i in range(len(train_data)):
        for j in range(len(train_data[i])):
            aux = list()
            aux_syn = list()
            aux_emg = list()
            for k in range(len(train_data[i][j])):
                aux = np.append(aux, train_data[i][j][k])
                aux_syn = np.append(aux_syn, train_syn[i][j][k])
                aux_emg = np.append(aux_emg, train_emg[i][j][k])
            if np.count_nonzero(np.isnan(aux)) == 0 and np.count_nonzero(np.isnan(aux_syn)) == 0 and np.count_nonzero(np.isnan(aux_emg)) == 0: # DISCARD EPs WITH ALL KINEMATICS OR ALL SYNERGIES AS NaN
                trn_dat.append(aux)  # size = Joints X bins
                trn_lab.append(train_given[i])
                trn_syn.append(aux_syn)  # size = Synergies X bins
                trn_emg.append(aux_emg)  # size = Electrodes X bins

    tst_dat = list()
    tst_emg = list()
    tst_lab = list()
    tst_syn = list()
    for i2 in range(len(test_data)):
        for j2 in range(len(test_data[i2])):
            aux2 = list()
            aux2_syn = list()
            aux2_emg = list()
            for k2 in range(len(test_data[i2][j2])):
                aux2 = np.append(aux2, test_data[i2][j2][k2])
                aux2_syn = np.append(aux2_syn, test_syn[i2][j2][k2])
                aux2_emg = np.append(aux2_emg, test_emg[i2][j2][k2])
            if np.count_nonzero(np.isnan(aux2)) == 0 and np.count_nonzero(np.isnan(aux2_syn)) == 0 and np.count_nonzero(np.isnan(aux2_emg)) == 0: # DISCARD EPs WITH ALL KINEMATICS OR ALL SYNERGIES AS NaN
                tst_dat.append(aux2)
                tst_lab.append(test_given[i2])
                tst_syn.append(aux2_syn)
                tst_emg.append(aux2_emg)
                
    # USING LOGISTIC REGRESSION
    log_model = LogisticRegression(penalty='elasticnet', C=0.2, solver='saga', max_iter=25000, multi_class='multinomial', l1_ratio=0.5)
    log_model.fit(trn_dat, trn_lab)
#     pred = log_model.predict_proba(tst_dat)

#     cl = log_model.classes_
#     hits = 0
#     for i in range(len(pred)):
#         if cl[np.argmax(pred[i])] == tst_lab[i]:
#             hits += 1
#     acc = round((hits/len(tst_lab))*100, 2)
#     total_acc.append(acc)
#     print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

#     #  USING GLMNET
#     # lbl = LabelEncoder()
#     # dum = np.float64(lbl.fit_transform(trn_lab))
#     #
#     # cvfit = cvglmnet(x=np.array(trn_dat), y=dum, family='multinomial', alpha=0.5)
#     # pred = cvglmnetPredict(cvfit, np.array(tst_dat), ptype='class')
#     #
#     # dum_res = np.float64(lbl.fit_transform(tst_lab))
#     # hits = 0
#     # for i in range(len(pred)):
#     #     if pred[i] == dum_res[i]:
#     #         hits += 1
#     # acc = round((hits/len(tst_lab))*100, 2)
#     # total_acc.append(acc)
#     # print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

#     # USING SVM
#     # clf = svm.SVC(C=25, kernel='poly', degree=3, decision_function_shape='ovo')  # 45.91
#     # clf.fit(np.array(trn_dat), trn_lab)
#     # pred = clf.predict(np.array(tst_dat))
#     # hits = 0
#     # for i in range(len(pred)):
#     #     if pred[i] == tst_lab[i]:
#     #         hits += 1
#     # acc = round((hits/len(tst_lab))*100, 2)
#     # total_acc.append(acc)
#     # print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

#     # USING ANN
#     # num_hidden_layers = 4
#     # layers = np.ceil(np.geomspace(len(trn_dat[0]), 3, num_hidden_layers+2)).astype(int)
#     # clf = MLPClassifier(solver='adam', alpha=20, hidden_layer_sizes=(layers[1:len(layers)-1]), random_state=1, max_iter=100000, activation='tanh')
#     # clf.fit(np.array(trn_dat), trn_lab)
#     # pred = clf.predict(np.array(tst_dat))
#     # hits = 0
#     # for i in range(len(pred)):
#     #     if pred[i] == tst_lab[i]:
#     #         hits += 1
#     # acc = round((hits/len(tst_lab))*100, 2)
#     # total_acc.append(acc)
#     # print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

#     # FOR MULTIPLE SOURCE CLASSIFICATION
#     trn_dat = list()
#     trn_lab = list()

#     for i in range(len(train_data)):
#         for j in range(len(train_data[i])):
#             aux = list()
#             aux_emg = list()
#             for k in range(len(train_data[i][j])):
#                 aux = np.append(aux, train_data[i][j][k])
#                 aux_emg = np.append(aux_emg, train_emg[i][j][k])
#             if np.count_nonzero(np.isnan(aux)) == 0 and np.count_nonzero(np.isnan(aux_emg)) == 0: # DISCARD EPs WITH ALL KINEMATICS OR ALL SYNERGIES AS NaN
#                 trn_lab.append(train_given[i])  
#                 res = np.concatenate((aux, aux_emg), axis=None)
#                 trn_dat.append(res)

#     tst_dat = list()
#     tst_lab = list()
#     for i2 in range(len(test_data)):
#         for j2 in range(len(test_data[i2])):
#             aux2 = list()
#             aux2_emg = list()
#             for k2 in range(len(test_data[i2][j2])):
#                 aux2 = np.append(aux2, test_data[i2][j2][k2])
#                 aux2_emg = np.append(aux2_emg, test_emg[i2][j2][k2])
#             if np.count_nonzero(np.isnan(aux2)) == 0 and np.count_nonzero(np.isnan(aux2_emg)) == 0: # DISCARD EPs WITH ALL KINEMATICS OR ALL SYNERGIES AS NaN
#                 res = np.concatenate((aux2, aux2_emg), axis=None)
#                 tst_dat.append(res)
#                 tst_lab.append(test_given[i2])
                
#     # USING LOGISTIC REGRESSION
#     log_model = LogisticRegression(penalty='elasticnet', C=0.2, solver='saga', max_iter=25000, multi_class='multinomial', n_jobs=-1, l1_ratio=0.5)
#     log_model.fit(trn_dat, trn_lab)
#     weights = log_model.coef_
#     # print("Size Weights: ", weights.shape)
#     # print("Classes: ", log_model.classes_)
    
    # WRITE WEIGHTS INTO A FILE
    file_name = "./PyData/" + family_to_select + "_Kin.csv"
    weights = log_model.coef_
    resh = weights.reshape((weights.shape[0], int(weights.shape[1]/num_bins), num_bins))
    mean_resh = np.mean(resh, axis=2)
    df = pd.DataFrame(mean_resh, index=log_model.classes_)
    df.to_csv(file_name, index=True, header=None, mode='a')
    print("FILE UPDATED")
# 
#     pred = log_model.predict_proba(tst_dat)
# 
#     cl = log_model.classes_
#     hits = 0
#     for i in range(len(pred)):
#         if cl[np.argmax(pred[i])] == tst_lab[i]:
#             hits += 1
#     acc = round((hits/len(tst_lab))*100, 2)
#     total_acc.append(acc)
#     print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

    #  USING GLMNET
    # lbl = LabelEncoder()
    # dum = np.float64(lbl.fit_transform(trn_lab))
    #
    # cvfit = cvglmnet(x=np.array(trn_dat), y=dum, family='multinomial', alpha=0.5)
    # pred = cvglmnetPredict(cvfit, np.array(tst_dat), ptype='class')
    #
    # dum_res = np.float64(lbl.fit_transform(tst_lab))
    # hits = 0
    # for i in range(len(pred)):
    #     if pred[i] == dum_res[i]:
    #         hits += 1
    # acc = round((hits/len(tst_lab))*100, 2)
    # total_acc.append(acc)
    # print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

    # USING SVM
    # clf = svm.SVC(C=25, kernel='poly', degree=3, decision_function_shape='ovo')  # 45.91
    # clf.fit(np.array(trn_dat), trn_lab)
    # pred = clf.predict(np.array(tst_dat))
    # hits = 0
    # for i in range(len(pred)):
    #     if pred[i] == tst_lab[i]:
    #         hits += 1
    # acc = round((hits/len(tst_lab))*100, 2)
    # total_acc.append(acc)
    # print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

    # USING ANN
    # num_hidden_layers = 4
    # layers = np.ceil(np.geomspace(len(trn_dat[0]), 3, num_hidden_layers+2)).astype(int)
    # clf = MLPClassifier(solver='adam', alpha=20, hidden_layer_sizes=(layers[1:len(layers)-1]), random_state=1, max_iter=100000, activation='tanh')
    # clf.fit(np.array(trn_dat), trn_lab)
    # pred = clf.predict(np.array(tst_dat))
    # hits = 0
    # for i in range(len(pred)):
    #     if pred[i] == tst_lab[i]:
    #         hits += 1
    # acc = round((hits/len(tst_lab))*100, 2)
    # total_acc.append(acc)
    # print("Hits: ", hits, " out of ", len(tst_lab), ". ", acc, "%")

# print("|| FAMILY: ", family_to_select, " || Mean accuracy after ", cv, " folds with ", num_bins, " bins per EP: ", round(np.mean(total_acc), 2), " % ||")
print("|| FAMILY: ", family_to_select,  " || Bins per EP: ", num_bins, " ||")
print("DONE!")

FILE UPDATED
FILE UPDATED
FILE UPDATED
|| FAMILY:  Ball  || Bins per EP:  20  ||
DONE!
